In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

try:
    SparkContext.getOrCreate().stop()
except:
    pass

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Test") \
    .getOrCreate()

print(spark.version)



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\socket.py", line 719, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
KeyboardInterrupt


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: org.apache.spark.SparkException: Only one SparkContext should be running in this JVM (see SPARK-2243).The currently running SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.wa

In [ ]:
# !curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2025-01.parquet

In [ ]:
df = spark.read.parquet("fhvhv_tripdata_2025-01.parquet")
df.show(5, False)

In [ ]:
df.schema

In [ ]:
partition_df = df.repartition(24)

In [ ]:
# partition_df.write.parquet('tripdata/2025/01')

In [ ]:
partition_df.printSchema()

In [ ]:
partition_df.select('PULocationID','DOLocationID','tips').filter(partition_df.tips > 0).show(5, False)

In [ ]:
from pyspark.sql import functions as F

def total_income(driver_pay,tips):
    if driver_pay is None:
        driver_pay = 0
    if tips is None:
        tips = 0
    return driver_pay + tips
total_income_udf = F.udf(total_income)
total_income_udf(partition_df.driver_pay, partition_df.tips) # 110

In [ ]:
partition_df \
    .withColumn('pickup_date', F.to_date(partition_df.pickup_datetime).cast("String")) \
    .withColumn('dropoff_date', F.to_date(partition_df.dropoff_datetime).cast("String")) \
    .withColumn('total_income', partition_df.driver_pay + partition_df.tips) \
    .select(
        'PULocationID',
        'DOLocationID',
        'pickup_date',
        'dropoff_date',
        'tips',
        'total_income'
    ) \
    .show(5, False)
